# Second Layer: Genesis of Non-Gaussianity

In [6]:
import sympy as sp
import symdl as sml

In this notebook, we first workout quartic coupling math tools, then derive the layer distribution

Let's start with quartic action:

$$S[z] = \frac{1}{2} \sum_{\alpha_1, \alpha_2 \in \mathcal{D}} g^{\alpha_1 \alpha_2} \sum_{i=1}^{n} z_{i; \alpha_1} z_{i; \alpha_2}
- \frac{1}{8} \epsilon \sum_{\alpha_1, \ldots, \alpha_4 \in \mathcal{D}} v^{(\alpha_1 \alpha_2)(\alpha_3 \alpha_4)} 
\sum_{i_1, i_2 = 1}^{n} z_{i_1; \alpha_1} z_{i_1; \alpha_2} z_{i_2; \alpha_3} z_{i_2; \alpha_4}$$

Note that the expression of quartic is differ from the book by $\epsilon$, this is because we need to take limit with respect to $\epsilon$ by `sympy.series`, and the order will be $O(\epsilon)$ instead of $O(v)$ in the book


In [7]:
i = sml.neuron_indices('i1:5')
j = sml.neuron_indices('j1:5')
alpha = sml.sample_indices('alpha1:5')
beta = sml.sample_indices('beta1:5')

g_inv = sml.NNIndexedBase('g', symmetries='full', idx_is_superscript = [0,1]) # action
g = sml.NNIndexedBase('g', symmetries='full')
v = sml.NNIndexedBase('v', symmetries=[(0,1), (2,3),([0,1],[2,3])], idx_is_superscript = True) # action
z = sml.NNIndexedBase('z', is_gaussian=True)

n = sp.Symbol("n", integer = True) # number of neouron
N_D = sp.Symbol("N_D", integer = True) # number of sample
eps = sp.Symbol("epsilon")

def take_eps_limit(expr: sp.Expr) -> sp.Expr:
    taylor: sp.Expr = sp.series(expr, x=eps, x0=0, n=2)
    return taylor

quartic_term = eps * (
    -sp.Rational(1, 8)
    * sp.Sum(
        v[beta[0], beta[1], beta[2], beta[3]]
        * sp.Sum(
            z[j[0], beta[0]] * z[j[0], beta[1]] * z[j[1], beta[2]] * z[j[1], beta[3]],
            (j[0], 1, n),
            (j[1], 1, n),
        ),
        (beta[0], 1, N_D),
        (beta[1], 1, N_D),
        (beta[2], 1, N_D),
        (beta[3], 1, N_D),
    )
)
quartic_term

-epsilon*Sum(v[beta1, beta2, beta3, beta4]*Sum(z[j1, beta1]*z[j1, beta2]*z[j2, beta3]*z[j2, beta4], (j1, 1, n), (j2, 1, n)), (beta1, 1, N_D), (beta2, 1, N_D), (beta3, 1, N_D), (beta4, 1, N_D))/8

Now let’s look at the following equation:

$$
\mathbb{E}\left[ F(z_{i_1;\alpha_1}, \ldots, z_{i_m;\alpha_m}) \right]
= \frac{ \int \left[ \prod_{i,\alpha} dz_{i;\alpha} \right] e^{-S(z)} 
    F(z_{i_1;\alpha_1}, \ldots, z_{i_m;\alpha_m}) }
  { \int \left[ \prod_{i,\alpha} dz_{i;\alpha} \right] e^{-S(z)} }
\\= 
\frac{
\left\langle\!\left\langle 
\exp\left\{
\frac{1}{8} \sum_{\beta_1, \ldots, \beta_4 \in \mathcal{D}} 
v^{(\beta_1 \beta_2)(\beta_3 \beta_4)} 
\sum_{j_1, j_2 = 1}^{n} 
z_{j_1; \beta_1} z_{j_1; \beta_2} z_{j_2; \beta_3} z_{j_2; \beta_4}
\right\} 
F(z_{i_1; \alpha_1}, \ldots, z_{i_m; \alpha_m})
\right\rangle\!\right\rangle_g
}{
\left\langle\!\left\langle 
\exp\left\{
\frac{1}{8} \sum_{\beta_1, \ldots, \beta_4 \in \mathcal{D}} 
v^{(\beta_1 \beta_2)(\beta_3 \beta_4)} 
\sum_{j_1, j_2 = 1}^{n} 
z_{j_1; \beta_1} z_{j_1; \beta_2} z_{j_2; \beta_3} z_{j_2; \beta_4}
\right\}
\right\rangle\!\right\rangle_g
}
\\
\textcolor{red}{=}
\left\langle\!\left\langle 
F(z_{i_1; \alpha_1}, \ldots, z_{i_m; \alpha_m})
\right\rangle\!\right\rangle_g
\\
+ \frac{1}{8} 
\sum_{\beta_1, \ldots, \beta_4 \in \mathcal{D}} 
v^{(\beta_1 \beta_2)(\beta_3 \beta_4)} 
\sum_{j_1, j_2 = 1}^{n} 
\left[
\left\langle\!\left\langle 
z_{j_1; \beta_1} z_{j_1; \beta_2} z_{j_2; \beta_3} z_{j_2; \beta_4} 
F(z_{i_1; \alpha_1}, \ldots, z_{i_m; \alpha_m})
\right\rangle\!\right\rangle_g 
\right.
\\
\left.
- 
\left\langle\!\left\langle 
z_{j_1; \beta_1} z_{j_1; \beta_2} z_{j_2; \beta_3} z_{j_2; \beta_4} 
\right\rangle\!\right\rangle_g
\left\langle\!\left\langle 
F(z_{i_1; \alpha_1}, \ldots, z_{i_m; \alpha_m}) 
\right\rangle\!\right\rangle_g
\right]
\\
+ \mathcal{O}(v^2)

$$

We’ll just work out the last equality (the red one) since the earlier steps are pretty straightforward.

In [8]:
F = sml.RandomSymbol("F")
Eg = sml.GaussianExpVal(g)

EF = Eg(sp.exp(-quartic_term) * F)/Eg(sp.exp(-quartic_term))
EF

⟨F*exp(epsilon*Sum(v[beta1, beta2, beta3, beta4]*Sum(z[j1, beta1]*z[j1, beta2]*z[j2, beta3]*z[j2, beta4], (j1, 1, n), (j2, 1, n)), (beta1, 1, N_D), (beta2, 1, N_D), (beta3, 1, N_D), (beta4, 1, N_D))/8)⟩_g/⟨exp(epsilon*Sum(v[beta1, beta2, beta3, beta4]*Sum(z[j1, beta1]*z[j1, beta2]*z[j2, beta3]*z[j2, beta4], (j1, 1, n), (j2, 1, n)), (beta1, 1, N_D), (beta2, 1, N_D), (beta3, 1, N_D), (beta4, 1, N_D))/8)⟩_g

In [9]:
EF = take_eps_limit(EF)
EF = sml.pull_sums_out_front(EF)
EF # result

⟨F⟩_g + epsilon*(Sum(⟨F*z[j1, beta1]*z[j1, beta2]*z[j2, beta3]*z[j2, beta4]⟩_g*v[beta1, beta2, beta3, beta4]/8, (j1, 1, n), (j2, 1, n), (beta1, 1, N_D), (beta2, 1, N_D), (beta3, 1, N_D), (beta4, 1, N_D)) + Sum(-⟨F⟩_g*⟨z[j1, beta1]*z[j1, beta2]*z[j2, beta3]*z[j2, beta4]⟩_g*v[beta1, beta2, beta3, beta4]/8, (j1, 1, n), (j2, 1, n), (beta1, 1, N_D), (beta2, 1, N_D), (beta3, 1, N_D), (beta4, 1, N_D))) + O(epsilon**2)

The result consist with book. 

Let's workout two-point correlator and four-point connected correlator

$\mathbb{E}\left[ z_{i_1; \alpha_1} \, z_{i_2; \alpha_2} \right]$ is:


In [10]:
Ezz = EF.subs(F, z[i[0], alpha[0]] * z[i[1], alpha[1]])
Ezz

⟨z[i1, alpha1]*z[i2, alpha2]⟩_g + epsilon*(Sum(⟨z[i1, alpha1]*z[i2, alpha2]*z[j1, beta1]*z[j1, beta2]*z[j2, beta3]*z[j2, beta4]⟩_g*v[beta1, beta2, beta3, beta4]/8, (j1, 1, n), (j2, 1, n), (beta1, 1, N_D), (beta2, 1, N_D), (beta3, 1, N_D), (beta4, 1, N_D)) + Sum(-⟨z[i1, alpha1]*z[i2, alpha2]⟩_g*⟨z[j1, beta1]*z[j1, beta2]*z[j2, beta3]*z[j2, beta4]⟩_g*v[beta1, beta2, beta3, beta4]/8, (j1, 1, n), (j2, 1, n), (beta1, 1, N_D), (beta2, 1, N_D), (beta3, 1, N_D), (beta4, 1, N_D))) + O(epsilon**2)

In [11]:
Ezz = sml.wick_contraction(Ezz)
Ezz

⟨z[i1, alpha1]*z[i2, alpha2]⟩_g + epsilon*(Sum((⟨z[i1, alpha1]*z[i2, alpha2]⟩_g*⟨z[j1, beta1]*z[j1, beta2]⟩_g*⟨z[j2, beta3]*z[j2, beta4]⟩_g + ⟨z[i1, alpha1]*z[i2, alpha2]⟩_g*⟨z[j1, beta1]*z[j2, beta3]⟩_g*⟨z[j1, beta2]*z[j2, beta4]⟩_g + ⟨z[i1, alpha1]*z[i2, alpha2]⟩_g*⟨z[j1, beta1]*z[j2, beta4]⟩_g*⟨z[j1, beta2]*z[j2, beta3]⟩_g + ⟨z[i1, alpha1]*z[j1, beta1]⟩_g*⟨z[i2, alpha2]*z[j1, beta2]⟩_g*⟨z[j2, beta3]*z[j2, beta4]⟩_g + ⟨z[i1, alpha1]*z[j1, beta1]⟩_g*⟨z[i2, alpha2]*z[j2, beta3]⟩_g*⟨z[j1, beta2]*z[j2, beta4]⟩_g + ⟨z[i1, alpha1]*z[j1, beta1]⟩_g*⟨z[i2, alpha2]*z[j2, beta4]⟩_g*⟨z[j1, beta2]*z[j2, beta3]⟩_g + ⟨z[i1, alpha1]*z[j1, beta2]⟩_g*⟨z[i2, alpha2]*z[j1, beta1]⟩_g*⟨z[j2, beta3]*z[j2, beta4]⟩_g + ⟨z[i1, alpha1]*z[j1, beta2]⟩_g*⟨z[i2, alpha2]*z[j2, beta3]⟩_g*⟨z[j1, beta1]*z[j2, beta4]⟩_g + ⟨z[i1, alpha1]*z[j1, beta2]⟩_g*⟨z[i2, alpha2]*z[j2, beta4]⟩_g*⟨z[j1, beta1]*z[j2, beta3]⟩_g + ⟨z[i1, alpha1]*z[j2, beta3]⟩_g*⟨z[i2, alpha2]*z[j1, beta1]⟩_g*⟨z[j1, beta2]*z[j2, beta4]⟩_g + ⟨z[i1, alpha

In [12]:
A, B, C, D = sml.wilds("A, B, C, D")
gaussian_rule = {Eg(z[A, B] * z[C, D]): sp.KroneckerDelta(A, C) * g[B, D]}
Ezz = sml.wild_subs(Ezz, gaussian_rule)
Ezz

KroneckerDelta(i1, i2)*g[alpha1, alpha2] + epsilon*(Sum((KroneckerDelta(i1, i2)*KroneckerDelta(j1, j2)*g[alpha1, alpha2]*g[beta1, beta3]*g[beta2, beta4] + KroneckerDelta(i1, i2)*KroneckerDelta(j1, j2)*g[alpha1, alpha2]*g[beta1, beta4]*g[beta2, beta3] + KroneckerDelta(i1, i2)*g[alpha1, alpha2]*g[beta1, beta2]*g[beta3, beta4] + KroneckerDelta(i1, j1)*KroneckerDelta(i2, j1)*g[alpha1, beta1]*g[alpha2, beta2]*g[beta3, beta4] + KroneckerDelta(i1, j1)*KroneckerDelta(i2, j1)*g[alpha1, beta2]*g[alpha2, beta1]*g[beta3, beta4] + KroneckerDelta(i1, j1)*KroneckerDelta(i2, j2)*KroneckerDelta(j1, j2)*g[alpha1, beta1]*g[alpha2, beta3]*g[beta2, beta4] + KroneckerDelta(i1, j1)*KroneckerDelta(i2, j2)*KroneckerDelta(j1, j2)*g[alpha1, beta1]*g[alpha2, beta4]*g[beta2, beta3] + KroneckerDelta(i1, j1)*KroneckerDelta(i2, j2)*KroneckerDelta(j1, j2)*g[alpha1, beta2]*g[alpha2, beta3]*g[beta1, beta4] + KroneckerDelta(i1, j1)*KroneckerDelta(i2, j2)*KroneckerDelta(j1, j2)*g[alpha1, beta2]*g[alpha2, beta4]*g[beta1, b

In [13]:
Ezz = sml.canonicalize_dummy_indices(sp.expand(Ezz))
Ezz = sml.pull_coef_out_sum(Ezz)
Ezz

KroneckerDelta(i1, i2)*g[alpha1, alpha2] + epsilon*Sum(KroneckerDelta(i1, j1)*KroneckerDelta(i2, j2)*KroneckerDelta(j1, j2)*g[alpha1, beta1]*g[alpha2, beta2]*g[beta3, beta4]*v[beta1, beta3, beta2, beta4], (beta1, 1, N_D), (beta2, 1, N_D), (beta3, 1, N_D), (beta4, 1, N_D), (j1, 1, n), (j2, 1, n)) + epsilon*Sum(KroneckerDelta(i1, j1)*KroneckerDelta(i2, j1)*g[alpha1, beta1]*g[alpha2, beta2]*g[beta3, beta4]*v[beta1, beta2, beta3, beta4], (beta1, 1, N_D), (beta2, 1, N_D), (beta3, 1, N_D), (beta4, 1, N_D), (j1, 1, n), (j2, 1, n))/2 + O(epsilon**2)

In [14]:
Ezz = sml.sum_kronecker_contract(Ezz)
Ezz

KroneckerDelta(i1, i2)*g[alpha1, alpha2] + epsilon*Sum(KroneckerDelta(i1, j1)*KroneckerDelta(i2, j1)*g[alpha1, beta1]*g[alpha2, beta2]*g[beta3, beta4]*v[beta1, beta2, beta3, beta4], (beta1, 1, N_D), (beta2, 1, N_D), (beta3, 1, N_D), (beta4, 1, N_D), (j1, 1, n), (j2, 1, n))/2 + epsilon*Sum(KroneckerDelta(i1, j1)*KroneckerDelta(i2, j1)*g[alpha1, beta1]*g[alpha2, beta2]*g[beta3, beta4]*v[beta1, beta3, beta2, beta4], (beta1, 1, N_D), (beta2, 1, N_D), (beta3, 1, N_D), (beta4, 1, N_D), (j1, 1, n)) + O(epsilon**2)

In [15]:
Ezz

KroneckerDelta(i1, i2)*g[alpha1, alpha2] + epsilon*Sum(KroneckerDelta(i1, j1)*KroneckerDelta(i2, j1)*g[alpha1, beta1]*g[alpha2, beta2]*g[beta3, beta4]*v[beta1, beta2, beta3, beta4], (beta1, 1, N_D), (beta2, 1, N_D), (beta3, 1, N_D), (beta4, 1, N_D), (j1, 1, n), (j2, 1, n))/2 + epsilon*Sum(KroneckerDelta(i1, j1)*KroneckerDelta(i2, j1)*g[alpha1, beta1]*g[alpha2, beta2]*g[beta3, beta4]*v[beta1, beta3, beta2, beta4], (beta1, 1, N_D), (beta2, 1, N_D), (beta3, 1, N_D), (beta4, 1, N_D), (j1, 1, n)) + O(epsilon**2)

The joint distribution of preactivations in the ﬁrst and second layers is:

$$p\left(z^{(2)}, z^{(1)} \mid \mathcal{D} \right) = p\left(z^{(2)} \mid z^{(1)} \right) p\left(z^{(1)} \mid \mathcal{D} \right)
$$

The marginal distribution of the second-layer preactivations can then be obtained by **marginalizing over** or **integrating out** the ﬁrst-layer
preactivations:

$$p\left(z^{(2)} \mid \mathcal{D} \right) = \int \left[ \prod_{i, \alpha} dz^{(1)}_{i;\alpha} \right] p\left(z^{(2)} \mid z^{(1)} \right) p\left(z^{(1)} \mid \mathcal{D} \right)
$$

- The ﬁrst-layer marginal distribution is:

$$p\left(z^{(1)} \mid \mathcal{D} \right) = \frac{1}{\left| 2\pi G^{(1)} \right|^{n_1 / 2}} \exp\left( -\frac{1}{2} \sum_{i=1}^{n_1} \sum_{\alpha_1, \alpha_2 \in \mathcal{D}} G^{\alpha_1 \alpha_2}_{(1)} z^{(1)}_{i; \alpha_1} z^{(1)}_{i; \alpha_2} \right)
$$

- The conditional distribution is:

$$p\left(z^{(2)} \mid z^{(1)} \right) = \frac{1}{\left| 2\pi \widehat{G}^{(2)} \right|^{n_2/2}} \exp\left( -\frac{1}{2} \sum_{i=1}^{n_2} \sum_{\alpha_1, \alpha_2 \in \mathcal{D}} \widehat{G}^{\alpha_1 \alpha_2}_{(2)} z^{(2)}_{i; \alpha_1} z^{(2)}_{i; \alpha_2} \right)$$

where $\widehat{G}^{(2)}_{\alpha_1 \alpha_2} \equiv C^{(2)}_b + C^{(2)}_W \frac{1}{n_1} \sum_{j=1}^{n_1} \sigma^{(1)}_{j; \alpha_1} \sigma^{(1)}_{j; \alpha_2}$ is *stochastic* second-layer metric

